# BERTopic

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 65.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.36-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.3 MB/s eta 0:00:00
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039172 sha256=8e0b60516c8361df60c33e892541fc2a2ff5fe57b6a012c81cae063f5d7d18e5
  Stored in di

In [3]:
!pip install bertopic[visualization]

In [4]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 47.7 MB/s eta 0:00:00


## 라이브러리 불러오기

In [5]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer
from bertopic import BERTopic

In [ ]:
reviews = pd.read_csv('/content/drive/MyDrive/윤호정 교수님 프로젝트/콜센터(식품) 메인 분석용 데이터.csv', encoding = 'utf-8')
reviews.head()

,IDX,발화자,발화문,카테고리,QA번호,QA여부,감성,인텐트 (대),인텐트 (중),인텐트 (소),...,수량(수치),크기,장소,조직,사람,시간,날짜,상품명,상담번호,상담내순번
0,1,c,컵밥 2만원 이상을 사면 무료 배송으로 아는데 여기가 죽도 추가로 몇 개 더 주문해...,식품,46175,q,m,배송,비용,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,컵밥|죽,46175,1
1,2,s,박스에 합포가 되지 않을 수도 있어 정확한 답변 드리지 못하는 점 양해 부탁드립니다.,식품,46175,a,m,배송,비용,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,46175,2
2,3,c,해마로 기준 6개 이상이고 볶음밥 기준으로는 50개 이상인데 해마로 케이준 1봉과 ...,식품,40543,q,m,배송,비용,질문,...,6.0,NaN,NaN,NaN,NaN,NaN,NaN,해마로|볶음밥,40543,1
3,4,s,일단 두 제품이 같이 담기면 최소 수량 부과인 6개 이상은 시스템 상 자동부과가 됩니다.,식품,40543,a,m,배송,비용,질문,...,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40543,2
4,5,c,마트 원데이는 무료 배송이라고 되있는데 왜 결제를 하려고 하니까 배송비 3000원이...,식품,15426,q,m,배송,비용,질문,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15426,1


In [ ]:
reviews.columns

Index(['IDX', '발화자', '발화문', '카테고리', 'QA번호', 'QA여부', '감성', '인텐트 (대)',
       ' 인텐트 (중)', '인텐트 (소)', '가격', '가격(수치)', '수량', '수량(수치)', '크기', '장소', '조직',
       '사람', '시간', '날짜', '상품명', '상담번호', '상담내순번'],
      dtype='object')

In [ ]:
utters = reviews[["발화문", "인텐트 (대)"]]
utters.head()

,발화문,인텐트 (대)
0,컵밥 2만원 이상을 사면 무료 배송으로 아는데 여기가 죽도 추가로 몇 개 더 주문해...,배송
1,박스에 합포가 되지 않을 수도 있어 정확한 답변 드리지 못하는 점 양해 부탁드립니다.,배송
2,해마로 기준 6개 이상이고 볶음밥 기준으로는 50개 이상인데 해마로 케이준 1봉과 ...,배송
3,일단 두 제품이 같이 담기면 최소 수량 부과인 6개 이상은 시스템 상 자동부과가 됩니다.,배송
4,마트 원데이는 무료 배송이라고 되있는데 왜 결제를 하려고 하니까 배송비 3000원이...,배송


In [ ]:
utters["인텐트 (대)"].value_counts()

제품          207857
배송          184069
행사           14384
포장            8704
결제            7922
교환|반품|환불      7104
웹사이트          2080
AS            1904
멤버십            288
Name: 인텐트 (대), dtype: int64

In [ ]:
utters['인텐트 (대)'].replace({'교환|반품|환불': '기타', '웹사이트': '기타', 'AS': '기타', '멤버십': '기타'}, inplace=True)

In [ ]:
# 변환 완료
utters["인텐트 (대)"].value_counts()

제품    207857
배송    184069
행사     14384
기타     11376
포장      8704
결제      7922
Name: 인텐트 (대), dtype: int64

In [ ]:
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }

def clean(text, punct, mapping):
    for p in mapping:
        text = text.replace(p, mapping[p])

    for p in punct:
        text = text.replace(p, f' {p} ')

    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}
    for s in specials:
        text = text.replace(s, specials[s])

    return text.strip()

def clean_str(text):
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)' # E-mail제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+' # URL제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '([ㄱ-ㅎㅏ-ㅣ]+)'  # 한글 자음, 모음 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '<[^>]*>'         # HTML 태그 제거
    text = re.sub(pattern=pattern, repl='', string=text)
    pattern = '[^\w\s\n]'         # 특수기호제거
    text = re.sub(pattern=pattern, repl='', string=text)
    text = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','', string=text)
    text = re.sub('\n', '.', string=text)
    return text

In [ ]:
cleaned_sentences = []

for index, row in utters.iterrows():
    intent = row['발화문']
    category = row['인텐트 (대)']

    punct_clean = clean(str(intent), punct, punct_mapping)
    clean_sentence = clean_str(punct_clean)
    print("정제된 문장 : ",clean_sentence, "\n", "대분류 : ", category)
    cleaned_sentences.append((clean_sentence, category))

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
정제된 문장 :  아그리고 혹시 금요일 착 가능할까요  
 대분류 :  배송
정제된 문장 :  금요일에 받으시도록 내일 목요일에 출고해 드릴게요  
 대분류 :  배송
정제된 문장 :  11일 주문건인데요 몇 시간 전 주문건인 이 상품은 아직 배송 준비 중이고 몇 시간 후 주문한 커피는 주소 변경요청하니 물류센터로 주문이 넘어갔다고 안 해주고 순차적으로 처리한 다더니 귀찮은 건 나 몰라라 하는 거 아닙니까  
 대분류 :  배송
정제된 문장 :  특별 행사 행사 관련 주문량 폭주로 물류로 내려간 송장 찾을 수가 없어 도움이 불가한 점 양해 바랍니다  
 대분류 :  배송
정제된 문장 :  다음 주에 배송 가능한가요  
 대분류 :  배송
정제된 문장 :  아직 구매전이신 듯한데 요구매하시고 배송받고 싶은 날짜배송 요청사항에 적어주시면 확인 후 발송해드리도록 할게요  
 대분류 :  배송
정제된 문장 :  12월 15일로 배송 변경해주시면 안 될까요  
 대분류 :  배송
정제된 문장 :  취소 후 재주문해주시면 됩니다  
 대분류 :  배송
정제된 문장 :  아이스박스에 합배송 부탁드려요 금요일까지 받을 수 있겠죠  
 대분류 :  배송
정제된 문장 :  출고는 빠르면 오늘 늦어도 익일까지 출고 진행됩니다  
 대분류 :  배송
정제된 문장 :  주문했는데 요 오늘 출고 가능한가요  
 대분류 :  배송
정제된 문장 :  6월 6일 현충일로 인한 택배 휴무로 인하여 6월 5일 식품은 집하되지 않았습니다  
 대분류 :  배송
정제된 문장 :  5일째 오지 않고 있어요   오늘은 밤이고 내일이면 6일   곧 일주일인데 식품인데 괜찮은가요  
 대분류 :  배송
정제된 문장 :  택배사 분실로 오늘 재발송드릴게요  
 대분류 :  배송
정제된 문장 :  5월 6일에 필요한 거라 5월 5일 1시에픽업요청했어요   5월 5일에 받아볼 수 있나요  
 대분류 :  배송
정제된 문장 :  5월 5일 1시 픽

In [ ]:
before_sep = pd.DataFrame(cleaned_sentences, columns = ["utterance", "category"])
before_sep

,utterance,category
0,컵밥 2만원 이상을 사면 무료 배송으로 아는데 여기가 죽도 추가로 몇 개 더 주문해...,배송
1,박스에 합포가 되지 않을 수도 있어 정확한 답변 드리지 못하는 점 양해 부탁드립니다,배송
2,해마로 기준 6개 이상이고 볶음밥 기준으로는 50개 이상인데 해마로 케이준 1봉과 ...,배송
3,일단 두 제품이 같이 담기면 최소 수량 부과인 6개 이상은 시스템 상 자동부과가 됩니다,배송
4,마트 원데이는 무료 배송이라고 되있는데 왜 결제를 하려고 하니까 배송비 3000원이...,배송
...,...,...
434307,지금 주문주신다면 내일 출고되어 토요일에 받아보실 수 있으세요 참고 하시어 이용...,배송
434308,지금 주문했는데요 내일 꼭 받아야 합니다 가능한지요,배송
434309,주문건은 오늘 발송됩니다 송장번호 조회는 저녁 6시30분 이후부터 가능하세요,배송
434310,지금 사면 수요일에 배송받을 수 있나요,배송


In [ ]:
data_product = before_sep[before_sep["category"] == "제품"]
data_delivery = before_sep[before_sep["category"] == "배송"]
data_event = before_sep[before_sep["category"] == "행사"]
data_package = before_sep[before_sep["category"] == "포장"]
data_payment = before_sep[before_sep["category"] == "결제"]
data_etc = before_sep[before_sep["category"] == "기타"]

In [ ]:
data_product.to_csv('/content/drive/MyDrive/윤호정 교수님 프로젝트/product.csv', encoding='utf-8')
data_delivery.to_csv('/content/drive/MyDrive/윤호정 교수님 프로젝트/delivery.csv', encoding='utf-8')
data_event.to_csv('/content/drive/MyDrive/윤호정 교수님 프로젝트/event.csv', encoding='utf-8')
data_package.to_csv('/content/drive/MyDrive/윤호정 교수님 프로젝트/package.csv', encoding='utf-8')
data_payment.to_csv('/content/drive/MyDrive/윤호정 교수님 프로젝트/payment.csv', encoding='utf-8')
data_etc.to_csv('/content/drive/MyDrive/윤호정 교수님 프로젝트/etc.csv', encoding='utf-8')



---


- 만약 정제된 데이터부터 사용하고 싶다면, 아래의 코드를 활성화

In [6]:
data_product = pd.read_csv('/content/drive/MyDrive/윤호정 교수님 프로젝트/product.csv', encoding='utf-8')
data_delivery = pd.read_csv('/content/drive/MyDrive/윤호정 교수님 프로젝트/delivery.csv', encoding='utf-8')
data_event = pd.read_csv('/content/drive/MyDrive/윤호정 교수님 프로젝트/event.csv', encoding='utf-8')
data_package = pd.read_csv('/content/drive/MyDrive/윤호정 교수님 프로젝트/package.csv', encoding='utf-8')
data_payment = pd.read_csv('/content/drive/MyDrive/윤호정 교수님 프로젝트/payment.csv', encoding='utf-8')
data_etc = pd.read_csv('/content/drive/MyDrive/윤호정 교수님 프로젝트/etc.csv', encoding='utf-8')

In [7]:
intent_product = data_product["utterance"].to_list()
intent_delivery = data_delivery["utterance"].to_list()
intent_event = data_event["utterance"].to_list()
intent_package = data_package["utterance"].to_list()
intent_payment = data_payment["utterance"].to_list()
intent_etc = data_etc["utterance"].to_list()

In [25]:
stopwords = pd.read_excel('/content/drive/MyDrive/윤호정 교수님 프로젝트/한국어 불용어 목록.xlsx')
stopwords_list = stopwords['한국어 불용어 목록'].tolist()

In [26]:
# 최신 불용어 확인
stopwords_list[-5:]

['대해', '있을까요', '아닌가요', '주신', '위해']

In [10]:
import konlpy
from konlpy.tag import Okt
okt = Okt()

In [11]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger

    def __call__(self, sent):
        sent = sent[:1000000]
        word_pos = self.tagger.pos(sent)
        result = [word for word, pos in word_pos if pos in ['Noun', 'Adjective'] and len(word) > 1]
        return result

In [12]:
custom_tokenizer = CustomTokenizer(Okt())

In [13]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, stop_words=stopwords_list, max_features=3000)

In [14]:
from scipy.cluster import hierarchy as sch

In [15]:
model = BERTopic(embedding_model="sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens", \
                 vectorizer_model=vectorizer,
                 nr_topics= 20,
                 top_n_words=10,
                 calculate_probabilities=True)

## Cateogry가 기타

In [ ]:
topics, probs = model.fit_transform(intent_etc)

.gitattributes:   0%|          | 0.00/574 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/527 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
linkage_function = lambda x: sch.linkage(x, 'single', optimal_ordering=True)
hierarchical_topics = model.hierarchical_topics(intent_etc, linkage_function=linkage_function)

100%|██████████| 18/18 [00:00<00:00, 155.64it/s]


In [ ]:
model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

In [ ]:
topics_to_merge = [[9, 5], [0, 2]]
model.merge_topics(intent_etc, topics_to_merge)

In [ ]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,5963,-1_상품_반품_교환_처리,"[상품, 반품, 교환, 처리, 확인, 주문, 환불, 제품, 배송, 구매]","[반품 입고 확인이 되어 환불 처리가 되었습니다 , 9 18일 날 출고된 상품..."
1,0,4470,0_반품_환불_처리_배송,"[반품, 환불, 처리, 배송, 교환, 취소, 확인, 신청, 주문, 상품]","[잘못된 상품이 와서 교환신청했는데 언제 반품 수거하러 오나요 , 이 상품이 왜 환..."
2,1,262,1_상품_출고_발송_확인,"[상품, 출고, 발송, 확인, 월요일, 제품, 주문, 배송, 해당, 택배]",[해당 상품은 배송 중 파손되어 당 사물류센터로 재입고된 것으로 확인되어 8월 11...
3,2,200,2_장난_쿠폰_교환_뭔가,"[장난, 쿠폰, 교환, 뭔가, 상세, 휴가, 있나요, 대체, 페이, 소리]","[지금 장난합니까 , 장난하시나요 , 장난합니까 ]"
4,3,111,3_식품_섭취_제품_다이어트,"[식품, 섭취, 제품, 다이어트, 때문, 식사, 용기, 식감, 제조, 시기]",[저희 고온어다이어트 도시락 용기는 안심포장용기로 공인기관의 인체 안전성 테스트를 ...
5,4,91,4_답변_질문_답글_문의,"[답변, 질문, 답글, 문의, 댓글, 이유, 안달, 대답, 후기, 없나요]","[왜 제 꺼만 답변 안하세요 , 제 글에는 왜 답변이 안 달리는지 , 왜 제 글만 ..."
6,5,44,5_감사_즐거운_좋은_명절,"[감사, 즐거운, 좋은, 명절, 정말, 지적, 사랑, 문의, 맛있게, 이해]","[문의 글 남겨주셔서 감사드려요 , 문의 감사드리구요 , 문의 감사드려요 ]"
7,6,41,6_쿠폰_복구_가능할_퍼센트,"[쿠폰, 복구, 가능할, 퍼센트, 옵션, 예상, 적용, 일시, 선택, 모카]",[현재 쿠폰 적용에 일시적으로 오류가 있는 것으로 확인됩니다 오전 10시 이후 ...
8,7,40,7_고구마_숙성_상하_수확,"[고구마, 숙성, 상하, 수확, 온도, 환경, 보관, 수분, 겨울, 습도]",[네 고구마 제품은 아임웰 고구마로 검색하시어 나오는 상품으로 구매해 주시면 되세요...
9,8,25,8_선택_에스프레소_해기_커피,"[선택, 에스프레소, 해기, 커피, 드립, 갈비, 분쇄, 원두, 만델링, 인도네시아]","[에스프레소를 만드는 에스프레소 머신이라면 분쇄도는 에스프레소로 선택해주세요 , 해..."


In [ ]:
model.visualize_barchart(top_n_topics=8)

## Cateogry가 결제

In [ ]:
topics, probs = model.fit_transform(intent_payment)

In [ ]:
linkage_function = lambda x: sch.linkage(x, 'single', optimal_ordering=True)
hierarchical_topics = model.hierarchical_topics(intent_payment, linkage_function=linkage_function)

100%|██████████| 18/18 [00:00<00:00, 154.01it/s]


In [ ]:
model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

In [ ]:
topics_to_merge = [[0, 2]]
model.merge_topics(intent_payment, topics_to_merge)

In [ ]:
topic_keywords = model.get_topic_info()

In [ ]:
model.visualize_barchart(top_n_topics=8)

## Cateogry가 포장

In [ ]:
topics, probs = model.fit_transform(intent_package)

In [ ]:
linkage_function = lambda x: sch.linkage(x, 'single', optimal_ordering=True)
hierarchical_topics = model.hierarchical_topics(intent_package, linkage_function=linkage_function)

100%|██████████| 1/1 [00:00<00:00, 100.58it/s]


In [ ]:
model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

In [ ]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,51,0_확인_샘플_사진_,"[확인, 샘플, 사진, , , , , , , ]","[확인해보겠습니다 , 확인해보겠습니다 , 확인해볼게요]"
1,1,8653,1_포장_박스_선물_배송,"[포장, 박스, 선물, 배송, 주문, 상품, 제품, 발송, 아이스팩, 구매]","[박스 포장인가요 , 두 개 주문하면 박스로 포장되서 오나요 , 배송 박스 포장되서..."


In [ ]:
model.visualize_barchart()

## Category가 행사

In [ ]:
topics, probs = model.fit_transform(intent_event)

In [ ]:
linkage_function = lambda x: sch.linkage(x, 'single', optimal_ordering=True)
hierarchical_topics = model.hierarchical_topics(intent_event, linkage_function=linkage_function)

100%|██████████| 18/18 [00:00<00:00, 142.22it/s]


In [ ]:
model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

In [ ]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,7470,-1_사은_주문_증정_구매,"[사은, 주문, 증정, 구매, 이벤트, 상품, 행사, 발송, 세트, 배송]","[10 1 증정 이벤트는 20개 주문하셨으면 총 22개가 발송됩니다 , 네 2세트..."
1,0,5026,0_사은_주문_발송_구매,"[사은, 주문, 발송, 구매, 행사, 상품, 확인, 증정, 이벤트, 배송]",[문의하신사은품은 옵션에 상품명과 기재되어 있는 사은품이 발송됩니다 제품 주문에...
2,1,632,1_확인_종료_이벤트_행사,"[확인, 종료, 이벤트, 행사, 박스, 진행, 개사, 이면, 해당, 적용]","[확인해볼게요, 확인해볼게요, 종료된 행사로 확인됩니다 ]"
3,2,258,2_양해_상품_사은_고객,"[양해, 상품, 사은, 고객, 재고, 판매, 소진, 현재, 해당, 변경]",[주문하신 상품에는 담요사은품이 적용되지 않았으며 현재 재고 모두 소진되어 양해 부...
4,3,214,3_선택_제품_수량_수령,"[선택, 제품, 수량, 수령, 구매, 이벤트, 증정, 진행, 문의, 주문]",[문의주 신제품 현재 3 1 이벤트 진행 중으로 수량 3개로 선택하여 주문 주실 ...
5,4,197,4_샘플_커피_제품_쉐이크,"[샘플, 커피, 제품, 쉐이크, 구매, 증정, 우려, 선택, 가지, 현미]",[샘플은 쿠키앤 크림 코코아 바나나 고구마 총 4가지로 구성되어 있습니다 아쉽게...
6,5,154,5_옵션_구매_상품_적용,"[옵션, 구매, 상품, 적용, 복수, 증정, 수량, 사은, 금액, 참고]",[복수 구매 시 랜덤 무드 등 ID당 1개 증정은 옵션에서 상품 구매 수량 2개 이...
7,6,117,6_모카_골드_맥심_제품,"[모카, 골드, 맥심, 제품, 스텐, 아메리카노, 커피, 텀블러, 블렌드, 아이스]","[모카골드 640개 총 160 4 박스 구매하면 텀블러 2개 오나요 , 구매 제품은..."
8,7,71,7_행사_진행_판매_이벤트,"[행사, 진행, 판매, 이벤트, 일요일, 가용, 시작, 기간, 금요일, 개월]","[언제 또 행사하실까요 , 행사는 3월 4일부터 행사되었습니다 , 행사 언제까지 진..."
9,8,44,8_텀블러_실리콘_하트_텀블,"[텀블러, 실리콘, 하트, 텀블, 있나요, 화이트골드, 거지, 스텐, 주지, 스트]","[돌려서 여는 텀블러입니다 , 왜 텀블러는 안주나요 , 텀블러 왜 안 보내주나요 ]"


In [ ]:
model.visualize_barchart(top_n_topics=8)

## Cateogry가 배송

In [16]:
print("데이터(행)의 개수 : ", len(intent_delivery))

데이터(행)의 개수 :  184069


In [39]:
import random

# 배송의 경우 데이터가 너무 크기 때문에 5만개로 랜덤 샘플링
intent_delivery_sample = random.sample(intent_delivery, 50000)
intent_delivery_sample[:5]

['왜 이렇게 늦나요 저번에는 금방 왔는데 이번에 왜 이렇게 오래 걸려요 ',
 '네 평일 오후 3시 이전 주문건은 당일 출고되요 ',
 '물류 쪽 누락으로 보여지며 누락된 상품은 오늘 재출고하여 내일 받아보실 수 있도록 처리할게요 ',
 'ooo 주문하 신상품 택배사로 잘 전달되어 내일 수령 가능하세요요즘 택배 물량이 너무 많아서 배송 알림 등록이 늦어져서요 내 일 점심 이후로 배송 예정 시간 수신 가능하세요 ',
 '쇼핑백도 꼭  보내주세요 하자 없는 상품으로 보내주시고   상품은 언제쯤 받아볼 수 있을까요 ']

In [40]:
topics, probs = model.fit_transform(intent_delivery_sample)

In [41]:
linkage_function = lambda x: sch.linkage(x, 'single', optimal_ordering=True)
hierarchical_topics = model.hierarchical_topics(intent_delivery_sample, linkage_function=linkage_function)

100%|██████████| 18/18 [00:00<00:00, 148.79it/s]


In [42]:
model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

In [43]:
topics_to_merge = [0, 1]
model.merge_topics(intent_delivery_sample, topics_to_merge)

In [44]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,28970,-1_주문_배송_발송_출고,"[주문, 배송, 발송, 출고, 오늘, 상품, 확인, 택배, 내일, 예정]","[주문했는데 오늘 출고가 될까요 , 네 15시 이전까지 주문하시면 오늘 발송하여 내..."
1,0,19630,0_배송_주문_발송_오늘,"[배송, 주문, 발송, 오늘, 출고, 상품, 월요일, 내일, 예정, 확인]","[주문 상품은 오늘 출고되어 내일 수령하십니다 , 오늘 주문 배송되나요 , 오늘 주..."
2,1,723,1_지연_물량_현재_택배,"[지연, 물량, 현재, 택배, 폭주, 증가, 출고, 양해, 발생, 상품]",[현재 코로나로 인하여 택배 사 물량이 증가함에 따라 배송이 지연이 되고 있는 것으...
3,2,139,2_상품_배송_오늘_금요일,"[상품, 배송, 오늘, 금요일, 내일, 제품, 주문, 발송, 시작, 있는지요]","[주문한 상품 언제 배송되나요 , 주문한 상품은 언제 배송되나요 , 제가 주문한 상..."
4,3,135,3_택배_사가_해외_회사,"[택배, 사가, 해외, 회사, 어딘가, 미국, 인지, 배송, 시나, 이용]","[택배사가 어디인가요 , 택배사가 어디인가요 , 택배택배사가 어디인가요 ]"
5,4,83,4_서울_발송_한글날_빈통,"[서울, 발송, 한글날, 빈통, 문건, 휴무, 회수, 완료, 순차, 당일]","[한글날 택배사 휴무로 인하여 발송은 월요일부터 발송됩니다 , 9일 한글날 연휴로 ..."
6,5,47,5_통운_이용_택배_옵션,"[통운, 이용, 택배, 옵션, 업체, 최근, 해당, 총파업, 비상, 가능하지]","[CJ대한통운 택배를 이용 중입니다 , 택배사는 CJ대한통운이용하고 있습니다 , C..."
7,6,43,6_해외_제주도_불가능합니다_아닙니다,"[해외, 제주도, 불가능합니다, 아닙니다, 불가, 배송, 지원, 마요네즈, 가능하지...","[해외 배송 시스템은 이용하지 않고 있습니다 , 저희는 해외 배송으로 무료 배송은 ..."
8,7,42,7_직원_마이_최대_이슈,"[직원, 마이, 최대, 이슈, 태풍, 죄송하게도, 사무실, 부탁드리며, 있으니, 인한]",[죄송하게도 현재 코로나 이슈로 인한 배송 지연과 태풍 마이삭 하이선으로 인해...
9,8,36,8_폭증_오류로_전산_시스템,"[폭증, 오류로, 전산, 시스템, 물량, 있으며, 정상, 있어, 코로나, 지연]",[현재 ooo 전산시스템 오류로 인해 상품 입고 가 정상적으로 되지 않고 있으며 o...


In [45]:
model.visualize_barchart(top_n_topics=8)

In [46]:
model.visualize_topics()

## Category가 제품

In [28]:
print("데이터(행)의 개수 : ", len(intent_product))

데이터(행)의 개수 :  207857


In [29]:
# 제품의 경우 데이터가 너무 크기 때문에 5만개로 랜덤 샘플링
intent_product_sample = random.sample(intent_product, 50000)
intent_product_sample[:5]

['문의게이너 73 섭취 방법 알려주세요 운동 전후 식사 사이 먹어주면 되나요 ',
 '장약한 사람한테는 안 맞나요 ',
 '김밥에 넣어서 계란 김밥 만들어 먹어도 되나요 ',
 '콜롬비아 슈프리모는 단일 품종입니다 ',
 '택배사 송장 출력 제한으로 일찍 마감될 것 같습니다   미리 받으시는 게 안전합니다 ']

In [31]:
topics, probs = model.fit_transform(intent_product_sample)

In [32]:
linkage_function = lambda x: sch.linkage(x, 'single', optimal_ordering=True)
hierarchical_topics = model.hierarchical_topics(intent_product_sample, linkage_function=linkage_function)

100%|██████████| 18/18 [00:00<00:00, 123.64it/s]


In [33]:
model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

In [35]:
topics_to_merge = [0, 2]
model.merge_topics(intent_product_sample, topics_to_merge)

In [36]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,31196,-1_제품_상품_보관_주문,"[제품, 상품, 보관, 주문, 섭취, 구매, 판매, 냉동, 유통, 기한]",[주문 주 신상품은 오늘 출고 예정이며 배송은 1 2일 소요되십니다 상...
1,0,14783,0_유통_기한_상품_주문,"[유통, 기한, 상품, 주문, 제품, 구매, 문의, 확인, 보관, 현재]","[문의하 신 상품의 유통기한은 2018 08월까지 인 상품이 있습니다 , 지금 주..."
2,1,2263,1_섭취_보관_제품_냉동,"[섭취, 보관, 제품, 냉동, 냉장, 권장, 커피, 때문, 식품, 사용]",[저희 제품은 냉동 보관제품이므로 냉장 보관에서는 여름에는 2 3일 지금 ...
3,2,823,2_고구마_수확_중국산_판매,"[고구마, 수확, 중국산, 판매, 고기, 국내, 마늘, 중국, 새우, 현재]",[퍽퍽하고 목메는 밤고구마는 수확초인 7 8월에 구매 시 맛보실 수 있으며...
4,3,221,3_단백질_비타민_운동_근육,"[단백질, 비타민, 운동, 근육, 섭취, 체중, 제품, 증가, 보충, 함유]",[체중 증가를 돕는 탄수화물과 근육 생성을 돕는 단백질이 모두 들어 있는 더 머슬게...
5,4,165,4_아닙니다_있지_없습니다_판매,"[아닙니다, 있지, 없습니다, 판매, 제품, 유기농, 포함, 스푼, 현재, 전혀]","[해당 상품은 판매 중이 아닙니다 , 네 아닙니다 , GMO 아닙니다 ]"
6,5,103,5_사과_바나나_과일_견과류,"[사과, 바나나, 과일, 견과류, 쥬스, 수확, 현미, 품종, 애플, 앙금]","[사과가 금값이네요 , 사과가 홍옥이 맞나요 , 자홍 사과 몇 개인가요 ]"
7,6,99,6_유기농_유전자_농약_친환경,"[유기농, 유전자, 농약, 친환경, 대두, 인증, 제품, 변형, 마크, 옥수수]","[유기농 맞습니까 , 유기농은 이니에요 , 유기농인가요 ]"
8,7,87,7_개월_제품_이내_가지,"[개월, 제품, 이내, 가지, 기한, 유통, 출고, 선택, 구성, 상품]",[생수유통기한은 제조일로부터 1년입니다 저희가 판매하는 상품의 제조일은 3개월 ...
9,8,59,8_종류_냉동고_참다랑어_나머지,"[종류, 냉동고, 참다랑어, 나머지, 보관, 일로, 가능해요, 일반, 기한, 냉장]",[일반 냉동고 보관시 참다랑어 종류는 3 4일 나머지 종류는 6 ...


In [37]:
model.visualize_barchart(top_n_topics=8)

In [38]:
model.visualize_topics()